# 01 — Coherence Engine Quick Start

This notebook demonstrates the core **Director-Class AI** consumer API:
the Coherence Engine that scores LLM outputs for factual and logical consistency.

In [ ]:
from director_ai.core import (
    CoherenceAgent,
    CoherenceScorer,
    SafetyKernel,
    MockGenerator,
    GroundTruthStore,
)

print(f"Director-AI loaded successfully")

## Coherence Scoring
The `CoherenceScorer` computes a dual-entropy score from NLI (logical) and RAG (factual) signals.

In [ ]:
scorer = CoherenceScorer()
store = GroundTruthStore()

# High coherence — consistent with ground truth
score_good = scorer.score("The sky is blue", store.retrieve_context("sky"))
print(f"Good claim score: {score_good}")  # Should be high

# Low coherence — contradicts reality
score_bad = scorer.score("The opposite is true about the sky", store.retrieve_context("sky"))
print(f"Bad claim score:  {score_bad}")   # Should be lower

## Safety Kernel
The `SafetyKernel` acts as a hardware interlock that can halt output.

In [ ]:
kernel = SafetyKernel(hard_limit=0.5)
print(f"Kernel active: {kernel.is_active}")
print(f"Check 0.8: {kernel.check(0.8)}")   # Pass
print(f"Check 0.3: {kernel.check(0.3)}")   # Fail → emergency stop
print(f"Kernel active after low score: {kernel.is_active}")

## Full Pipeline
The `CoherenceAgent` orchestrates generator → scorer → kernel.

In [ ]:
agent = CoherenceAgent()
result = agent.process("What color is the sky?")

print(f"Output:     {result.output}")
print(f"Coherence:  {result.coherence}")
print(f"Approved:   {result.approved}")
print(f"Reason:     {result.reason}")